# Creating the "TRANSCRIPT" dataset based on transcriptomics

Version 1.0.0 (December 28th 2022). Please run the notebook "FEATURELESS-v1.0.0_dataset.ipynb" beforehand.

## Libraries

In [1]:
import numpy as np
import pandas as pd
import subprocess as sb
import os

from time import sleep
from tqdm import tqdm
import requests
import pickle
import json

import sys
sys.path.insert(0, "../../src/")

import utils
import paths_global
import data_processing

from joblib import Parallel, delayed
from multiprocessing import cpu_count
njobs=min(max(1,cpu_count()-2),3)

## Local paths

In [2]:
## Where database files are stored
print('root_folder="%s"' % paths_global.root_folder)
## Where intermediary files are stored
print('data_folder="%s"' % paths_global.data_folder)

root_folder="/media/kali/1b80f30d-2803-4260-a792-9ae206084252/Code/M30/data/"
data_folder="/media/kali/1b80f30d-2803-4260-a792-9ae206084252/RECeSS/cfdr/data/"


In [3]:
transcript_folder = paths_global.data_folder+"TRANSCRIPT_v1.0.0/"
sb.Popen(["mkdir", "-p", transcript_folder])
## Where TRANSCRIPT dataset files are stored
print('transcript_folder="%s"' % transcript_folder)

transcript_folder="/media/kali/1b80f30d-2803-4260-a792-9ae206084252/RECeSS/cfdr/data/TRANSCRIPT_v1.0.0/"


## Drug and disease identifiers

In [4]:
assert os.path.exists(paths_global.data_folder+"drugbankid2drugname.pck")
with open(paths_global.data_folder+"drugbankid2drugname.pck", "rb") as f:
    di_drugbankid2drugname = pickle.load(f)
    
assert os.path.exists(paths_global.data_folder+"omimid2diseasename.pck")
with open(paths_global.data_folder+"omimid2diseasename.pck", "rb") as f:
    di_omimid2diseasename = pickle.load(f)
    
cids_file = paths_global.data_folder+"medgenid2diseasename.pck"
if (not os.path.exists(cids_file)):
    di_medgenid2diseasename = {}
else:
    with open(cids_file, "rb") as f:
        di_medgenid2diseasename = pickle.load(f)
        
pubchem_file = paths_global.data_folder+"pubchemid2drugname.pck"
if (not os.path.exists(pubchem_file)):
    di_pubchemid2drugname = {}
else:
    with open(pubchem_file, "rb") as f:
        di_pubchemid2drugname = pickle.load(f)

## I. Matrix A : $N_S \times N_D$ of drug-disease associations

In [5]:
A = pd.read_csv(paths_global.data_folder+"FEATURELESS_v1.0.0/all_ratings.csv", index_col=0)
A

,C1851649,C0042133,C5193005,C2676676,C1704272,C4722327,C1858361,C2676676.1,C4310232,C0029456,...,C0242770,C1880129,C0022661,C0236792,C1135191,C0149516,C1835407,C0016667,C0039445,C5203670
657181,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DB00010,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5311128,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DB00017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5311065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5917,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
442872,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
442021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4843,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
ratings_A = utils.matrix2ratings(A, "ind_id", "drug_id", "rating")

print("Sparsity = "+str(utils.compute_sparsity(A))+"%")
utils.print_dataset(ratings_A, "ind_id", "drug_id", "rating")
ratings_A.T

Sparsity = 0.35070326199346175%
Ndrugs=1599	Ndiseases=1599
8658 positive	320 negative	2547823 unknown matchings


,0,1,2,3,4,5,6,7,8,9,...,8968,8969,8970,8971,8972,8973,8974,8975,8976,8977
ind_id,C1851649,C0042133,C5193005,C2676676,C1704272,C4722327,C1858361,C0034013,C0014175,C1858361,...,C0014544,C0014544,C0014544,C0014544,C0014544,C0014544,C0014544,C0014544,C0014544,C5203670
drug_id,657181,657181,657181,657181,657181,657181,657181,657181,657181,DB00010,...,2310,492405,4375,3352,3373,5917,442872,442021,4843,DB12466
rating,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0


## II. Build matrix P : $N_F \times N_D$ of disease features

Here, features are disease "differential" phenotypes, that is vectors of genewise expression changes due to disease (by comparing differential expression between patient and healthy groups). We use [CREEDS](https://maayanlab.cloud/CREEDS/) (free access, without registration), which reports manually curated disease phenotypes and crowd-sourced phenotypes (see [paper](https://www.nature.com/articles/ncomms12846)).

### II.a. Conversion of genes into their corresponding human orthologs

Sometimes the phenotypes are computed on animal models, hence the need to convert (one-to-one) orthologs from that species to humans. Here, we only consider mice models outside of humans.
- go to https://www.ensembl.org/biomart/martview/
- select in "Dataset": "Mouse genes (GRCm39)"
- "Filters": None
- "Attributes": 
	Gene > "Gene Stable ID", "Gene name"
	Homologues > Human > "Human gene stable ID", "Human homology type", "Human gene name"
- select "Results" and "Go"

In [7]:
## Where the orthologs files are stored
print('orthologs_folder="%s"' % paths_global.orthologs_folder)

orthologs_folder="/media/kali/1b80f30d-2803-4260-a792-9ae206084252/Code/M30/data/orthologs/"


In [8]:
mouse_human = pd.read_csv(paths_global.orthologs_folder+"mouse_human_matchings.tsv", sep='\t', header=0, index_col=None)
mouse_human = mouse_human.loc[mouse_human["Human homology type"]=="ortholog_one2one"][["Gene name","Human gene name"]]
mouse_human.index = mouse_human["Gene name"]
mouse_human = mouse_human.to_dict()["Human gene name"]

### II.b. Retrieval of differential phenotypes

In [9]:
## Where the CREEDS files are stored
print('creeds_folder="%s"' % paths_global.creeds_folder)

creeds_folder="/media/kali/1b80f30d-2803-4260-a792-9ae206084252/Code/M30/data/CREEDS/"


In [10]:
if (not os.path.exists(transcript_folder+"all_diseases.csv")):
    P_manual = {}
    for k in ["manual"]:
        fname=creeds_folder+"disease_signatures-"+type_di[k]+"1.0.json"
        ii = 1
        while (True):
            content = iterator_json(fname, ii)
            if (str(content)=="None"):
                break
            disease_cid = content["umls_cui"]
            organism = content["organism"]
            print((disease_cid, organism))
            ii += 1
            if (organism not in ["human", "mouse"]):
                continue
            if (disease_cid in A.columns and disease_cid not in P_manual):
                if (organism == "human"):
                    sig = dict(content['down_genes']+content['up_genes'])
                elif (organism == "mouse"):
                    sig = dict([[mouse_human[g],gi] for g,gi in content['down_genes']+content['up_genes'] if (g in mouse_human)])
                P_manual.setdefault(disease_cid, sig)
                
    P_auto = {}
    seen_diseases = []
    seen_diseases = seen_diseases[:-1]
    for k in ["automatic"]:
        fname=creeds_folder+"disease_signatures-"+type_di[k]+"1.0.json"
        ii = 1
        while (True):
            content = iterator_json(fname, ii)
            if (str(content)=="None"):
                print("End of file")
                break
            disease_cid = []   
            ii += 1
            organism = content["organism"]
            if (not "disease_name" in content):
                continue
            if (not content["disease_name"] in seen_diseases):
                print((ii-1,organism, content["disease_name"]))
            if (organism not in ["mouse", "human"] and str(organism)!=str(list(sorted(['human', 'mouse'])))):
                continue
            if (not content["disease_name"] in seen_diseases):
                seen_diseases.append(content["disease_name"])
                for disease in content["disease_name"].split("|"):
                    unspecified_diseases = ["Syndrome", "incurable diseases","Virus Diseases"]
                    unspecified_diseases += ["Possessed","Disease","Disease Progression","disease transmission"]
                    if (disease in unspecified_diseases):
                        ## unspecified diseases
                        continue
                    try:
                        cid = get_concept_id(disease)
                        if (cid in A.columns and cid not in P_auto):
                            disease_cid.append(cid)
                    except:
                        if (disease == "Seryl-tRNA synthetase, mitochondrial"):
                            disease = "Seryl-tRNA synthetase"
                        elif (disease == "Alzheimer's Disease Pathway KEGG"):
                            disease = "Alzheimer Disease"
                        elif (disease == "Metastatic Renal Cell Cancer"):
                            disease = "Renal Cell Cancer"
                        elif (disease == "1-acylglycerol-3-phosphate O-acyltransferase ABHD5"):
                            disease = "ABHD5"
                        elif (disease == "S-adenosyl-L-methionine"):
                            ## unspecified
                            continue
                        elif (disease == "metaplastic cell transformation"):
                            ## unspecified
                            continue
                        elif (disease == "Lipoma-preferred partner"):
                            disease = "Lipoma"
                        elif (disease == "lipophosphoglycan"):
                            continue
                        elif (disease == "Refractory anaemia with excess blasts"):
                            disease = "Refractory cytopenias"
                        else:
                            print(disease)
                            raise ValueError
                            continue
                        cid = get_concept_id(disease)
                        if (cid in A.columns and cid not in P_auto):
                            disease_cid.append(cid)
            if (len(disease_cid) == 0):
                continue
            for dcid in disease_cid:
                if ((organism == "human") or ("human" in organism)):
                    sig = dict(content['down_genes']+content['up_genes'])
                elif (organism == "mouse"):
                    sig = dict([[mouse_human[g],gi] for g,gi in content['down_genes']+content['up_genes'] if (g in mouse_human)])
                P_auto.setdefault(dcid, sig)
    
    ## Merge the two datasets (favouring manually curated signatures over the other ones)
    from copy import deepcopy
    P = deepcopy(P_auto)
    P.update(P_manual)
    
    pd.DataFrame(P).fillna(0.).to_csv(transcript_folder+"all_diseases.csv")
    
P = pd.read_csv(transcript_folder+"all_diseases.csv", index_col=0)
P

,C2936783,C2239176,C3553462,C0035235,C0032285,C0010346,C0009324,C0029408,C0001973,C3495559,...,C1260899,C1527336,C0275804,C0017168,C0003615,C0003872,C0014544,C0040028,C0040034,C0018802
USMG5,-0.158146,-0.158146,0.000000,0.000000,0.0,0.038415,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.022347,0.0,0.0,0.000000
SLC26A2,-0.131811,-0.131811,0.000000,0.000000,0.0,-0.075141,-0.112273,0.0,0.0,0.0,...,0.0,-0.014750,0.0,0.000000,-0.036328,0.0,0.000000,0.0,0.0,0.000000
GUCA2A,-0.130771,-0.130771,0.000000,0.000000,0.0,-0.101133,-0.167974,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,-0.026553,0.0,0.000000,0.0,0.0,0.000000
AQP8,-0.126526,-0.126526,0.000000,0.000000,0.0,-0.035313,-0.085438,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,-0.033508,0.0,0.000000,0.0,0.0,0.000000
CEACAM6,-0.125712,-0.125712,0.050276,0.050276,0.0,0.118054,0.000000,0.0,0.0,0.0,...,0.0,0.018521,0.0,-0.017911,-0.014329,0.0,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SPRYD7,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,-0.010207
ALKBH7,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,-0.010192
SLC41A3,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,-0.009935
TRIM54,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.017070


In [11]:
"%d/%d (%.2f perc.)" % (len([x for x in A.columns if (x in P.columns)]), A.shape[1], len([x for x in A.columns if (x in P.columns)])*100/A.shape[1])

'144/1599 (9.01 perc.)'

In [12]:
"%d/%d (%.2f perc.)" % (len([x for x in P.columns if (x in A.columns)]), P.shape[1], len([x for x in P.columns if (x in A.columns)])*100/P.shape[1])

'144/144 (100.00 perc.)'

## III. Build matrix S : $N_F \times N_S$ of drug features

### III.a. Using [CREEDS](https://maayanlab.cloud/CREEDS/) dataset

Here, features are drug signatures, that is vectors of genewise expression changes due to treatment (by comparing differential expression between treated and untreated groups). We use [CREEDS](https://maayanlab.cloud/CREEDS/) (free access, without registration), which reports manually curated and crowd-sourced drug signatures (see [paper](https://www.nature.com/articles/ncomms12846)). Contrary to [LINCS](https://lincsproject.org/LINCS/tools/workflows/find-the-best-place-to-obtain-the-lincs-l1000-data) L1000 data (see [paper](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5990023/)), those drug signatures are computed on GEO datasets, which might avoid the caveat of using measures from immortalized cells, and to rely on the computational inference for most genes. Similarly, whenever necessary, we convert gene names to their one-to-one orthologs.

In [13]:
## Where the orthologs files are stored
print('orthologs_folder="%s"' % paths_global.orthologs_folder)
## Where the CREEDS files are stored
print('creeds_folder="%s"' % paths_global.creeds_folder)

orthologs_folder="/media/kali/1b80f30d-2803-4260-a792-9ae206084252/Code/M30/data/orthologs/"
creeds_folder="/media/kali/1b80f30d-2803-4260-a792-9ae206084252/Code/M30/data/CREEDS/"


In [14]:
if (not os.path.exists(transcript_folder+"all_drugs.csv")):

    A = pd.read_csv(paths_global.data_folder+"FEATURELESS_v1.0.0/all_ratings.csv", engine="python", index_col=0)

    type_di = {"manual": "v", "automatic": "p"}
    fs = ["disease_signatures", "single_drug_perturbations"]
    base_url = "https://maayanlab.cloud/CREEDS/download/"

    sb.call("mkdir -p "+paths_global.creeds_folder, shell=True)
    for f in fs:
        for kt in type_di:
            fname = f+"-"+type_di[kt]+"1.0.json"
            if (not os.path.exists(paths_global.creeds_folder+fname)):
                sb.call("wget -O "+paths_global.creeds_folder+fname+" "+base_url+fname, shell=True)

    def iterator_json(fname, idi):
        cmd = "cat "+fname+" | sed 's/\\}/\\n/g' | head -n"+str(idi+1)+" | tail -n1"
        res = sb.check_output(cmd, shell=True).decode("utf-8")[:-1]
        if (len(res) == 0):
            return None
        res = res[1:-1]+"\"}"
        content = json.loads(res)
        return content

    S_manual = {}
    for k in ["manual"]:
        fname=creeds_folder+"single_drug_perturbations-"+type_di[k]+"1.0.json"
        ii = 1
        while (True):
            content = iterator_json(fname, ii)
            if (str(content)=="None"):
                print("End of file")
                break
            drug_cid = content["pubchem_cid"]
            organism = content["organism"]
            print((ii,drug_cid, organism))
            ii += 1
            if (organism not in ["human", "mouse"]):
                continue
            if (str(drug_cid)=="None"):
                continue
            if (drug_cid in A.index and drug_cid not in S_manual):
                if (organism == "human"):
                    sig = dict(content['down_genes']+content['up_genes'])
                elif (organism == "mouse"):
                    sig = dict([[mouse_human[g],gi] for g,gi in content['down_genes']+content['up_genes'] if (g in mouse_human)])
                S_manual.setdefault(drug_cid, sig)

    def iterator_json(fname, idi):
        cmd = "cat "+fname+" | sed 's/\\}/\\n/g' | head -n"+str(idi+1)+" | tail -n1"
        res = sb.check_output(cmd, shell=True).decode("utf-8")[:-1]
        if (len(res) == 0):
            return None
        res = res[1:-1]+"\"}"
        try:
            content = json.loads(res)
            return content
        except:
            ## sanitize...
            res_ = res.split("drug_name\": \"")[0]+"drug_name\": \""
            res_ += "".join("".join("".join("".join("".join("".join(res.split("drug_name\": \"")[-1].split("\"")[0].split("{")).split("}")).split("[")).split("]")).split("\'")).split("\\"))
            res_ += "\""+res.split("drug_name\": \"")[-1].split("\"")[-1]
            res = res_
            try:
                content = json.loads(res)
                return content
            except:
                return 0

    S_auto = {}
    seen_drugs = []
    for k in ["automatic"]:
        fname=creeds_folder+"single_drug_perturbations-"+type_di[k]+"1.0.json"
        ii = 1
        while (True):
            content = iterator_json(fname, ii)
            if (str(content)=="None"):
                ## end of file
                print("End of file")
                break
            ii += 1
            if (str(content)=="0"):
                ## ill-formated 
                continue
            if (not "organism" in content):
                continue
            organism = content["organism"]
            if (not "drug_name" in content):
                continue
            if (not content["drug_name"] in seen_drugs):
                print((ii-1,organism, content["drug_name"]))
            drug_cid = []
            if (not content["drug_name"] in seen_drugs):
                seen_drugs.append(content["drug_name"])
                cids = utils.get_pubchem_id(content["drug_name"].split("|"))
                drug_cid = [cid for cid in cids if (cid in A.index and cid not in S_auto)]
            if (len(drug_cid) == 0):
                continue
            for dcid in drug_cid:
                if ((organism == "human") or ("human" in organism)):
                    sig = dict(content['down_genes']+content['up_genes'])
                elif (organism == "mouse"):
                    sig = dict([[mouse_human[g],gi] for g,gi in content['down_genes']+content['up_genes'] if (g in mouse_human)])
                S_auto.setdefault(dcid, sig)

    ## Merge the two datasets (favouring manually curated signatures over the other ones)
    from copy import deepcopy
    S = deepcopy(S_auto)
    S.update(S_manual)
    pd.DataFrame(S).fillna(0.)

    pd.DataFrame(S).fillna(0.).to_csv(transcript_folder+"all_drugs.csv")

S = pd.read_csv(transcript_folder+"all_drugs.csv", index_col=0)
S

,36314,11354606,679,2554,126941,3033,444795,5757,19649,6279,...,149096,10836,5379,446220,57469,3410,5754,6322,31401,5394
IGFBP3,-0.305380,-0.099444,0.0,0.0,0.0,0.0,0.01587,0.0,0.0,0.0,...,0.0,0.0,-0.012214,0.000000,-0.015497,-0.048162,-0.229562,0.0,0.0,0.150907
ADIRF,-0.158368,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
THBS2,-0.134381,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.017583,0.000000,0.054800,-0.018192,0.0,0.0,0.000000
C10orf10,-0.128742,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
AQP1,-0.128104,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EHBP1L1,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,-0.016950
C19orf60,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,-0.014467
TBX2,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,-0.013415
ZNF395,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.027301


### III.b. Using [LINCS L1000](https://maayanlab.cloud/Harmonizome/resource/LINCS+L1000+Connectivity+Map)

In order to populate further matrix S, one can get profiles from LINCS L$1000$ Level $3$ data, and apply Characteristic Direction on them to get signatures similar to the ones in CREEDS.

In [15]:
## Where LINCS L1000 files are stored
print('lincs_folder="%s"' % paths_global.lincs_folder)

lincs_folder="/media/kali/1b80f30d-2803-4260-a792-9ae206084252/Code/M30/NetworkOrientedRepurposingofDrugs/lincs/"


In [16]:
A = pd.read_csv(paths_global.data_folder+"FEATURELESS_v1.0.0/all_ratings.csv", engine="python", index_col=0)
P = pd.read_csv(transcript_folder+"all_diseases.csv", engine="python", index_col=0)
S = pd.read_csv(transcript_folder+"all_drugs.csv", engine="python", index_col=0)

other_drugs = [x for x in A.index if (str(x) not in S.columns)]
print(len(other_drugs))
## reduce the number of genes to ease computations?
var_thres=0
genes = list(S.loc[S.var(axis=1) > var_thres].index)
genes += list(P.loc[P.var(axis=1) > var_thres].index)
genes = list(set(genes))
len(genes)

1456


15744

Convert gene names into EntrezGene CIDs and EntrezGene CIDs into LINCS L1000 gene symbols using package NORDic

In [17]:
from NORDic.UTILS.LINCS_utils import *
from NORDic.UTILS.utils_data import *

## Convert gene names into EntrezGene CID
entrezgene_fname=transcript_folder+"entrezgenes_ids.csv"
if (not os.path.exists(entrezgene_fname)):
    probes = request_biodbnet(genes, "Gene Symbol and Synonyms", "Gene ID", 9606, chunksize=100)
    probes.to_csv(entrezgene_fname)
probes = pd.read_csv(entrezgene_fname,index_col=0)

print("%d/%d" % (probes.shape[0], len(genes)))

other_genes = list(probes[probes["Gene ID"]=="-"].index)
entrezgene_fname=transcript_folder+"entrezgenes_ids_all.csv"
if (not os.path.exists(entrezgene_fname)):
    #list_genes = [data_processing.get_biomart("%d/%d" % (x_id+1,len(other_genes)), x) for x_id, x in enumerate(other_genes)]
    list_genes = Parallel(n_jobs=njobs, backend='loky')(delayed(data_processing.get_biomart)("%d/%d" % (x_id+1,len(other_genes)), x) for x_id, x in enumerate(other_genes))
    other_probes = pd.DataFrame(list_genes, index=other_genes)
    other_probes.to_csv(entrezgene_fname)
other_probes = pd.read_csv(entrezgene_fname,index_col=0)
other_probes.columns = ["Gene ID"]
print("%d/%d" % (other_probes[other_probes["Gene ID"]!="-"].shape[0], len(other_genes)))

probes = pd.concat((probes.loc[[g for g in genes if ((g.upper() not in other_genes) and (g in probes.index))]], other_probes), axis=0)

if (len(list(probes[probes["Gene ID"]=="-"].index))>0):
    print("Not found genes:")
    print("%d/%d" % (len(list(probes[probes["Gene ID"]=="-"].index)), len(genes)))
probes = probes[probes["Gene ID"]!="-"]

model_genes = list(probes.index)

## convert EntrezGene ids back to (LINCS L1000) gene symbols
if (not os.path.exists(transcript_folder+"entrezid2symbols.csv")):
    pert_inames = [None]*len(model_genes)
    entrez_ids = [None]*len(model_genes)
    for ig, g in enumerate(model_genes):
        endpoint="genes"
        method="filter"
        all_entrezid = str(probes.loc[g]["Gene ID"]).split("; ")
        for entrezid in all_entrezid:
            params = {"where":{"entrez_id": str(entrezid)},"fields":["gene_symbol"]}
            request_url = build_url(endpoint, method, params=params, user_key=user_key)
            try:
                data = post_request(request_url, quiet=True, stime=0)
            except:
                continue
            if (len(data)==0):
                continue
            else:
                pert_inames[ig] = data[0]["gene_symbol"]
                entrez_ids[ig] = entrezid
                if (pert_inames[ig]==g):
                    break
        print("\t".join([g, str(pert_inames[ig]), str(ig+1), str(len(model_genes))]))
    pert_iname_ids = [ip for ip,p in enumerate(pert_inames) if (p is not None)]
    pd.DataFrame([[pert_inames[i] for i in pert_iname_ids], [entrez_ids[i] for i in pert_iname_ids]], columns=[model_genes[i] for i in pert_iname_ids], index=["Gene Symbol", "Entrez ID"]).T.to_csv(sigs_data_folder+"entrezid2symbols.csv")
pert_df = pd.read_csv(transcript_folder+"entrezid2symbols.csv", index_col=0)
pert_inames = list(pert_df["Gene Symbol"])
entrez_ids = list(pert_df["Entrez ID"])
model_genes = list(pert_df.index)

15744/15744
9/902
Not found genes:
893/15744


In [18]:
print("%.2f perc. (N=%d)" % (len([g for g in S.index if (g in pert_df.index)])*100/S.shape[0],len([g for g in S.index if (g in pert_df.index)])))
print("%.2f perc. (N=%d)" % (len([g for g in P.index if (g in pert_df.index)])*100/P.shape[0],len([g for g in P.index if (g in pert_df.index)])))

93.66 perc. (N=10024)
87.47 perc. (N=12038)


Convert missing DrugBank drug names into PubChem CIDs and get associated signatures from LINCS L1000

In [19]:
if (not os.path.exists(transcript_folder+"save_signatures.csv")):
    other_drugs_converted = [di_drugbankid2drugname.get(d, d) for d in other_drugs]
    inv_map = {v: k for k, v in di_pubchemid2drugname.items()}
    for idx, d in enumerate(other_drugs):
        dx = other_drugs_converted[idx]
        try:
            other_drugs_converted[idx] = int(dx) if (int(dx) in di_pubchemid2drugname) else inv_map.get(dx, None)
        except:
            other_drugs_converted[idx] = inv_map.get(dx, None)
    other_drugs_converted = [dx for dx in other_drugs_converted if (dx is not None)]
    "%.2f perc. (N=%d)" % (len(other_drugs_converted)*100/len(other_drugs),len(other_drugs_converted))

    signatures = get_profiles(other_drugs_converted, entrez_ids, path_to_lincs=path_to_lincs, nsigs=2, verbose=0)
    signatures.to_csv(transcript_folder+"save_signatures.csv")
    
signatures = pd.read_csv(transcript_folder+"save_signatures.csv", index_col=0)
ids = [pert_df.iloc[list(pert_df["Entrez ID"]).index(int(idx)),:]["Gene Symbol"] for idx in signatures.index]
signatures.index = ids
signatures

,5311128,5311498,493570,445354,1054,54687,5324346,5362129,82153,3958,...,5281078,16129706,2310,4375,3352,3373,5917,442872,442021,4843
A2M,0.004211,-0.007953,0.012509,-0.051398,-0.009290,0.004976,0.040918,0.002669,-0.003704,0.010427,...,-0.011196,-0.004524,0.022839,0.034118,0.000014,0.027960,-0.005206,0.020435,-0.003454,-0.019129
NAT1,-0.002460,0.020545,-0.021153,0.005485,0.002446,-0.031114,-0.021160,0.019177,0.029274,-0.002868,...,-0.013530,0.007315,-0.006189,0.000281,-0.008344,-0.002534,0.030247,-0.005044,-0.019232,-0.003018
SERPINA3,0.001860,0.012869,-0.009555,-0.003553,0.000099,0.001667,-0.009011,0.023065,0.031211,-0.019435,...,-0.030425,0.008625,-0.000381,0.019200,-0.055678,-0.023840,0.008077,0.000588,0.016774,0.003450
AADAC,0.011732,-0.000800,-0.013999,0.014293,-0.003196,-0.001094,-0.002626,-0.009416,-0.013727,0.002335,...,-0.007428,0.015931,0.003481,-0.013923,0.003363,-0.003875,0.003461,-0.010424,-0.005539,0.009355
AAMP,0.000900,0.007262,0.001338,0.009476,0.012429,0.000898,-0.009574,0.007500,-0.001397,0.011513,...,-0.004937,0.017714,-0.007305,0.002826,-0.001552,-0.003753,-0.001625,-0.004519,0.007522,0.013908
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
JMJD7,-0.004519,-0.008332,-0.004328,0.008560,-0.003966,0.001151,-0.006685,0.002405,-0.001554,-0.001448,...,0.006941,0.002554,-0.010250,-0.002952,0.000698,0.002259,0.002246,0.007328,0.008599,-0.007155
TIMM23,-0.002968,-0.001522,0.003604,-0.003464,0.001232,0.011251,-0.003626,-0.011817,0.010872,-0.002438,...,-0.007431,-0.000748,-0.002906,-0.002370,0.005608,-0.004449,-0.001422,-0.009624,-0.008588,0.002467
ZNF783,-0.001060,-0.009225,-0.002874,-0.005501,-0.007082,-0.000083,0.000620,0.005173,0.000937,0.004329,...,-0.002663,0.002190,-0.008245,0.003452,-0.003703,-0.007072,-0.002508,0.004237,0.002847,-0.007339
THAP9-AS1,-0.011316,0.011730,0.008664,0.000761,0.012618,-0.004360,-0.005032,-0.011883,-0.006719,0.008573,...,-0.016185,0.006754,-0.006451,-0.002437,-0.000758,-0.009862,0.017386,-0.005379,0.001778,0.017479


In [20]:
if (not os.path.exists(transcript_folder+"all_drugs_+LINCS.csv")):
    S_LINCS = S[[c for c in S.columns if (c not in signatures.columns)]].join(signatures, how="outer")
    S_LINCS.fillna(0).to_csv(transcript_folder+"all_drugs_+LINCS.csv")
    
S_LINCS = pd.read_csv(transcript_folder+"all_drugs_+LINCS.csv", index_col=0)
S_LINCS

,36314,11354606,679,2554,126941,3033,444795,5757,19649,6279,...,5281078,16129706,2310,4375,3352,3373,5917,442872,442021,4843
A1BG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
A1CF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.001028,-0.000363,0.004807,-0.000522,-0.000997,0.008423,0.002083,0.005748,0.001114,0.014441
A2M,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.011196,-0.004524,0.022839,0.034118,0.000014,0.027960,-0.005206,0.020435,-0.003454,-0.019129
AAAS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.001392,-0.001070,-0.009262,0.001435,-0.001956,0.003047,0.001742,0.004964,0.012373,0.006155
AACS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.003157,-0.003778,-0.015237,0.007205,-0.012258,-0.006826,-0.003272,-0.001598,0.001768,0.007927
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYG11B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ZYX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.008635,0.012023,-0.010802,-0.011957,0.007483,-0.000572,-0.003034,-0.007464,0.006834,-0.009079
ZZEF1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.000688,-0.001199,0.000477,-0.005147,-0.016044,-0.003332,-0.000163,-0.001941,-0.005947,0.006733
ZZZ3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.010314,0.003458,-0.009071,0.011935,0.011555,-0.004139,0.010807,0.000995,0.007324,0.003029


## IV. Merge A, P and S to get the final "TRANSCRIPT" dataset

Disease ids: Concept IDs, drug ids: DrugBank or PubChem CIDs. Build matrices A (ratings) drugs $\times$ diseases, P (disease features) disease features $\times$ diseases, S (drug features) drug features $\times$ drugs.

In [21]:
A = pd.read_csv(paths_global.data_folder+"FEATURELESS_v1.0.0/all_ratings.csv", engine="python", index_col=0)
P = pd.read_csv(transcript_folder+"all_diseases.csv", engine="python", index_col=0)
S = pd.read_csv(transcript_folder+"all_drugs_+LINCS.csv", engine="python", index_col=0)

A_TRANSCRIPT = A[P.columns].loc[[c for c in S.columns if (c in A.index)]]
SP = S.join(P, how="inner").fillna(0)
S_TRANSCRIPT = SP[S.columns]
P_TRANSCRIPT = SP[P.columns]

S_TRANSCRIPT.to_csv(transcript_folder+"items.csv")
P_TRANSCRIPT.to_csv(transcript_folder+"users.csv")
A_TRANSCRIPT.to_csv(transcript_folder+"ratings_mat.csv")

ratings_A = utils.matrix2ratings(A_TRANSCRIPT, "ind_id", "drug_id", "rating")
print("Sparsity = "+str(utils.compute_sparsity(A_TRANSCRIPT))+"%")
print("%d drug features %d disease features" % (S_TRANSCRIPT.shape[0], P_TRANSCRIPT.shape[0]))
utils.print_dataset(ratings_A, "ind_id", "drug_id", "rating")
ratings_A.T

Sparsity = 0.7606199770378874%
10811 drug features 10811 disease features
Ndrugs=558	Ndiseases=118
773 positive	181 negative	64890 unknown matchings


,0,1,2,3,4,5,6,7,8,9,...,944,945,946,947,948,949,950,951,952,953
ind_id,C0020538,C1140680,C0007131,C0036220,C0236780,C0036341,C0014544,C2936783,C0033860,C0149925,...,C0014544,C0023418,C0014544,C0014544,C0014544,C0014544,C0014544,C0014544,C0014544,C0014544
drug_id,36314,36314,36314,36314,2554,2554,2554,126941,126941,126941,...,9915886,5281078,2310,4375,3352,3373,5917,442872,442021,4843
rating,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0
